<a href="https://colab.research.google.com/github/1405043-kd/QuoraInsincereQuestionClassification/blob/master/QuoraTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip

--2019-02-04 06:37:43--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2019-02-04 06:37:43--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  20.8MB/s    in 1m 59s  

2019-02-04 06:39:42 (17.5 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [0]:
import zipfile
zipRef=zipfile.ZipFile('glove.840B.300d.zip','r')
zipRef.extractall()
zipRef.close()

In [0]:
import pandas as pd
trainDf=pd.read_csv("/content/gdrive/My Drive/colabData/MlProject/train.csv")

In [0]:
import numpy as np 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

In [0]:
def readEmbeddings(embeddingsFile):
    
    tempDict={}
    f=open(embeddingsFile)
    for line in tqdm(f):
        values=line.split(" ")
        word=values[0]
        vect=np.asarray(values[1:],dtype='float32')
        tempDict[word]=vect
    f.close()
   
    return tempDict
embeddingsDict=readEmbeddings('glove.840B.300d.txt')

vectDim=300
numOfVect=30
len(embeddingsDict['the'])

2196017it [02:56, 12419.12it/s]


300

In [0]:
def textToArr(txt):
    zeroVect=np.zeros(vectDim)
    txt=txt[:-1].split()[:numOfVect]
    embeds=[embeddingsDict.get(x,zeroVect) for x in txt]
    embeds+=[zeroVect]*(numOfVect-len(embeds)) 
    return np.array(embeds)

textToArr("what is quora?").shape

(30, 300)

In [0]:
trainDf=trainDf[:300000]
trainDf, valDf = train_test_split(trainDf, test_size=0.2, random_state=42)
valDf, testDf = train_test_split(valDf, test_size=0.5, random_state=42)
print(trainDf.shape)
print(testDf.shape)
print(valDf.shape) 

(240000, 3)
(30000, 3)
(30000, 3)


In [0]:
#valVects = np.array([textToArr(x) for x in tqdm(valDf["question_text"])])
valY = np.array(valDf["target"])
tempLst=[]
for x in tqdm(valDf["question_text"]):
    try:
        tempLst.append(textToArr(x))
    except:
        pass
valVects=np.array(tempLst)

100%|██████████| 30000/30000 [00:02<00:00, 13758.67it/s]


In [0]:
#testVects = np.array([textToArr(x) for x in tqdm(testDf["question_text"])])
#testY = np.array(testDf["target"])

testY = np.array(testDf["target"])
tempLst1=[]
for x in tqdm(testDf["question_text"]):
    try:
        tempLst1.append(textToArr(x))
    except:
        pass
testVects=np.array(tempLst1)

100%|██████████| 30000/30000 [00:02<00:00, 13742.53it/s]


In [0]:
batchSize = 128

def batchGenerate(train_df):
    numBatches = math.ceil(len(train_df) / batchSize)
    while True: 
        train_df = train_df.sample(frac=1.)   
        for i in range(numBatches):
            texts = train_df.iloc[i*batchSize:(i+1)*batchSize, 1]
            textArr = np.array([textToArr(text) for text in texts])
            yield textArr, np.array(train_df["target"][i*batchSize:(i+1)*batchSize])

In [0]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Dropout
#from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [0]:
from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred): 
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred): 
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(30, 300)))
#model.add(Dropout(0.2))
model.add(Bidirectional(CuDNNLSTM(64)))
#model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',f1])

#plot_model(model,to_file='modelplot.png',show_shapes=True,show_layer_names=True)


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 30, 128)           187392    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 128)               99328     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 286,849
Trainable params: 286,849
Non-trainable params: 0
_________________________________________________________________


In [0]:
bg = batchGenerate(trainDf)
model.fit_generator(bg, epochs=20, steps_per_epoch=1000, validation_data=(valVects, valY), verbose=True) 

Epoch 1/20
1000/1000 [==============================] - 60s 60ms/step - loss: 0.1339 - acc: 0.9497 - f1: 0.4546 - val_loss: 0.1288 - val_acc: 0.9475 - val_f1: 0.5889
Epoch 2/20
1000/1000 [==============================] - 55s 55ms/step - loss: 0.1196 - acc: 0.9527 - f1: 0.5272 - val_loss: 0.1170 - val_acc: 0.9547 - val_f1: 0.5610
Epoch 3/20
1000/1000 [==============================] - 56s 56ms/step - loss: 0.1142 - acc: 0.9553 - f1: 0.5650 - val_loss: 0.1137 - val_acc: 0.9562 - val_f1: 0.5444
Epoch 4/20
1000/1000 [==============================] - 56s 56ms/step - loss: 0.1085 - acc: 0.9579 - f1: 0.5936 - val_loss: 0.1120 - val_acc: 0.9559 - val_f1: 0.5905
Epoch 5/20
1000/1000 [==============================] - 56s 56ms/step - loss: 0.1088 - acc: 0.9575 - f1: 0.5951 - val_loss: 0.1109 - val_acc: 0.9566 - val_f1: 0.6071
Epoch 6/20
1000/1000 [==============================] - 55s 55ms/step - loss: 0.1011 - acc: 0.9602 - f1: 0.6351 - val_loss: 0.1103 - val_acc: 0.9563 - val_f1: 0.6078
Epoc

In [0]:
score = model.evaluate(testVects, testY, batch_size=1024, verbose=1)

40000/40000 [==============================] - 3s 87us/step


In [0]:
print(model.metrics_names[0],":",score[0])
print(model.metrics_names[1],":",score[1])
print(model.metrics_names[2],":",score[2])

loss : 0.14998370048999787
acc : 0.9546
f1 : 0.6104958294868469
